In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def sample_first_prows(data, perc):
    return data.head(int(len(data)*(perc)))

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#data = data.loc[0: : 5]
data.dropna(inplace=True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
1,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [7]:
# frame as supervised learning
reframed = series_to_supervised_mimo(data, 1, 1)
reframed.head()

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
1,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
5,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [8]:
cols = reframed.columns
cols

Index(['use [kW](t-1)', 'gen [kW](t-1)', 'Dishwasher [kW](t-1)',
       'Furnace 1 [kW](t-1)', 'Furnace 2 [kW](t-1)', 'Home office [kW](t-1)',
       'Fridge [kW](t-1)', 'Wine cellar [kW](t-1)', 'Garage door [kW](t-1)',
       'Kitchen 12 [kW](t-1)', 'Kitchen 14 [kW](t-1)', 'Kitchen 38 [kW](t-1)',
       'Barn [kW](t-1)', 'Well [kW](t-1)', 'Microwave [kW](t-1)',
       'Living room [kW](t-1)', 'Solar [kW](t-1)', 'temperature(t-1)',
       'humidity(t-1)', 'visibility(t-1)', 'apparentTemperature(t-1)',
       'pressure(t-1)', 'windSpeed(t-1)', 'windBearing(t-1)',
       'precipIntensity(t-1)', 'dewPoint(t-1)', 'precipProbability(t-1)',
       'use [kW](t)', 'gen [kW](t)', 'Dishwasher [kW](t)', 'Furnace 1 [kW](t)',
       'Furnace 2 [kW](t)', 'Home office [kW](t)', 'Fridge [kW](t)',
       'Wine cellar [kW](t)', 'Garage door [kW](t)', 'Kitchen 12 [kW](t)',
       'Kitchen 14 [kW](t)', 'Kitchen 38 [kW](t)', 'Barn [kW](t)',
       'Well [kW](t)', 'Microwave [kW](t)', 'Living room [kW](t)',

In [9]:
df_forecats_columns = ['use [kW](t)', 'gen [kW](t)', 'Dishwasher [kW](t)', 'Furnace 1 [kW](t)',
       'Furnace 2 [kW](t)', 'Home office [kW](t)', 'Fridge [kW](t)',
       'Wine cellar [kW](t)', 'Garage door [kW](t)', 'Kitchen 12 [kW](t)',
       'Kitchen 14 [kW](t)', 'Kitchen 38 [kW](t)', 'Barn [kW](t)',
       'Well [kW](t)', 'Microwave [kW](t)', 'Living room [kW](t)',
       'Solar [kW](t)', 'temperature(t)', 'humidity(t)', 'visibility(t)',
       'apparentTemperature(t)', 'pressure(t)', 'windSpeed(t)',
       'windBearing(t)', 'precipIntensity(t)', 'dewPoint(t)',
       'precipProbability(t)']

In [10]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            df_test = ttest.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            df_original = ttest.loc[:,'use [kW](t)':'precipProbability(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [11]:
num_components_pca = 3
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [12]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 16796
--------------------
training window 33592
--------------------
training window 50388
--------------------
training window 67184
--------------------
training window 83980
--------------------
training window 100776
--------------------
training window 117572
--------------------
training window 134368
--------------------
training window 151164
--------------------
training window 167960
--------------------
training window 184756
--------------------
training window 201552
--------------------
training window 218348
--------------------
training window 235144
--------------------
training window 251940
--------------------
training window 268736
--------------------
training window 285532
--------------------
training window 302328
--------------------
training window 319124
--------------------
training window 335920
--------------------
training window 352716
--------------------
training window 36951

In [13]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [14]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,use [kW](t),0.377,0.188,89.418,0.693,13.549,0.056
1,gen [kW](t),0.034,0.023,inf,0.914,47.248,0.071
2,Dishwasher [kW](t),0.053,0.011,inf,-2.508,93.437,0.039
3,Furnace 1 [kW](t),0.060,0.026,50.486,0.776,18.684,0.074
4,Furnace 2 [kW](t),0.064,0.032,26.429,0.801,11.453,0.106
5,Home office [kW](t),0.021,0.012,19.771,0.902,10.260,0.038
6,Fridge [kW](t),0.023,0.008,66.945,0.907,20.983,0.040
7,Wine cellar [kW](t),0.010,0.005,40.397,0.959,16.221,0.054
8,Garage door [kW](t),0.011,0.001,5.078,0.119,1.950,0.036
9,Kitchen 12 [kW](t),0.005,0.001,inf,0.451,14.954,0.014


In [15]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_kaggle_sh.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [16]:
num_components_pca = 8 #12 #8
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [17]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 16796
--------------------
training window 33592
--------------------
training window 50388
--------------------
training window 67184
--------------------
training window 83980
--------------------
training window 100776
--------------------
training window 117572
--------------------
training window 134368


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 151164
--------------------
training window 167960


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 184756
--------------------
training window 201552
--------------------
training window 218348
--------------------
training window 235144
--------------------
training window 251940
--------------------
training window 268736
--------------------
training window 285532
--------------------
training window 302328
--------------------
training window 319124
--------------------
training window 335920
--------------------
training window 352716
--------------------
training window 369512
--------------------
training window 386308
--------------------
training window 403104
--------------------
training window 419900
--------------------
training window 436696
--------------------
training window 453492
--------------------
training window 470288
--------------------
training window 487084


In [21]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,use [kW](t),0.545,0.394,233.071,0.422,25.704,0.081
1,gen [kW](t),0.058,0.045,inf,0.732,59.656,0.120
2,Dishwasher [kW](t),0.164,0.108,inf,0.091,96.549,0.120
3,Furnace 1 [kW](t),0.119,0.087,226.294,0.045,49.533,0.152
4,Furnace 2 [kW](t),0.111,0.083,81.901,0.268,27.412,0.190
5,Home office [kW](t),0.080,0.057,91.750,0.327,34.910,0.153
6,Fridge [kW](t),0.064,0.047,504.832,0.259,51.842,0.112
7,Wine cellar [kW](t),0.047,0.036,254.703,0.082,52.897,0.266
8,Garage door [kW](t),0.014,0.005,38.821,-0.059,14.241,0.041
9,Kitchen 12 [kW](t),0.008,0.004,inf,0.198,36.893,0.026


In [22]:
final_measures_pca.to_csv(r'pca_mo_ensfts_kaggle_sh_.csv', index = False, header=True)

In [24]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_kaggle_sh_2.csv', index = False, header=True)